This notebook is just a demonstration on how to build RNN in PyTorch.

Dataset used for this demonstration is: Fashion Mnist dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.autograd import Variable
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms 
from torch.optim import Adam
import numpy as np

In [2]:
def accuracy(preds, y_true):
    '''
    Use this function to check accuracy of a model trained.
    
    :param: preds - predictions generated by neural network
    :param: y_true - true/real labels for each sample in the dataset
    '''
    correct = 0 
    assert len(preds) == len(y_true)
    
    for i in range(len(preds)):
        if np.argmax(preds[i]) == y_true[i]:
            correct += 1
    return correct / len(preds)

In [3]:
#hyperparams
learning_rate = 0.003
num_epochs = 6
batch_size = 100
rnn_size = 128
seq_len = 28
in_vector_size = 28
num_layers = 2
num_classes = 10

In [4]:
#Download and transform FashionMNIST training set
train_data = FashionMNIST(root='./data/', 
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)

In [5]:
#Unpack and transform FashionMNIST test set
test_data = FashionMNIST(root='./data/', 
                          train=False, 
                          transform=transforms.ToTensor(), 
                          download=False)

In [6]:
#Create a training set loader - DataLoader will help us with batching a dataset
train_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=2)

In [7]:
#Create a test set loader
test_loader = torch.utils.data.DataLoader(test_data, 
                                           batch_size=batch_size, 
                                           shuffle=False, 
                                           num_workers=2)

#### Create RNN in PyTorch

In [8]:
class SimpleRNN(nn.Module):
    
    def __init__(self, input_size, num_of_classes, rnn_size, number_of_layers):
        '''
            Init function of the SimpleRNN class helps to setup everything that we need for rnn.
            
            :param: input_size - size of input vector
            :param: rnn_size - number of units/neurons in the RNN
            :param: number_of_layers - number of RNN layers
            :param: num_of_classes - number of different classes in a dataset (e.p. MNIST has 10 classes)
        '''
        super(SimpleRNN, self).__init__()
        
        self.hidden_units = rnn_size
        self.num_layers = number_of_layers
        
        self.rnn = nn.LSTM(input_size, rnn_size, number_of_layers, batch_first=True)
        self.out_layer = nn.Linear(self.hidden_units, num_of_classes)
        
    def forward(self, X):
        '''
        This method is called for networks forward-prop.
        
        :param: X - batch of data from a dataset
        '''
     
        #Create starting states for the LSTM layer
        h0 = Variable(torch.zeros(self.num_layers, X.size(0), self.hidden_units).cuda())
        c0 = Variable(torch.zeros(self.num_layers, X.size(0), self.hidden_units).cuda())
        
        out, _ = self.rnn(X, (h0, c0))
        
        out = f.softmax(self.out_layer(out[:, -1, :]))
        return out

In [9]:
#Create RNN network object
rnn = SimpleRNN(in_vector_size, num_classes, rnn_size, num_layers)

In [10]:
#If you have avaliable cuda supported GPU card use this command to run the network on GPU instead of CPU
rnn.cuda()

SimpleRNN(
  (rnn): LSTM(28, 128, num_layers=2, batch_first=True)
  (out_layer): Linear(in_features=128, out_features=10)
)

In [11]:
#Create object of the cross entropy loss function
criterion = nn.CrossEntropyLoss()

In [12]:
#Create object of the Adam optimizer
optimizer = Adam(rnn.parameters(), lr=learning_rate)

#### Create a training loop

In [13]:
for epoch in range(num_epochs):
    epoch_accuracy = []
    epoch_loss = []
    
    for images, labels in train_loader:
        
        X_batch = Variable(images.view(-1, seq_len, in_vector_size)).cuda()
        y_batch = Variable(labels).cuda()
        
        optimizer.zero_grad()
        preds = rnn(X_batch)
        epoch_accuracy.append(accuracy(preds.cpu().data.numpy(), y_batch.cpu().data.numpy()))
        loss = criterion(preds, y_batch)
        epoch_loss.append(loss.cpu().data.numpy())
        loss.backward()
        optimizer.step()
        
    print("Epoch: {}/{}".format(epoch+1, num_epochs), 
          " | Epoch loss: {}".format(np.mean(epoch_loss)), 
          " | Epoch accuracy: {}".format(np.mean(epoch_accuracy)))

c:\users\lukaa\appdata\local\continuum\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/6  | Epoch loss: 1.7887852191925049  | Epoch accuracy: 0.67635
Epoch: 2/6  | Epoch loss: 1.7101843357086182  | Epoch accuracy: 0.7513166666666666
Epoch: 3/6  | Epoch loss: 1.677836298942566  | Epoch accuracy: 0.7833833333333333
Epoch: 4/6  | Epoch loss: 1.660081386566162  | Epoch accuracy: 0.8012666666666667
Epoch: 5/6  | Epoch loss: 1.652994155883789  | Epoch accuracy: 0.8082833333333334
Epoch: 6/6  | Epoch loss: 1.6279258728027344  | Epoch accuracy: 0.8331833333333334
